In [1]:
import pandas as pd
import numpy as np
time_lst = [-2, -1,  1,  2,3,4] 
df = pd.read_csv("./df_saki_timeseries_feature.csv")
df = df[df["time"].isin(time_lst)]

df_ = df.drop(["dataset",'groupHPD'],axis=1)
df_.head(2)

,stay_id,time,bicarbonate,sodium,temperature,po2,fio2,dbp,creatinine,aniongap,...,chloride,glucose,urineoutput,spo2,hemoglobin,crea_divide_basecrea,sbp,mbp,lactate,ph
0,5,-2,21.0,140.0,35.8,162.0,41.0,70.2341,0.6554,11.5,...,105.0,76.200002,20.0,98.000002,14.28686,1.03,114.899234,84.676544,1.0,7.33
1,5,-1,21.0,140.0,35.8,162.0,41.0,70.2341,0.6554,11.5,...,105.0,76.200002,20.0,98.000002,14.28686,1.03,114.899234,84.676544,1.0,7.33


In [2]:
# df = pd.read_csv("./df_saki_timeseries_feature.csv")
# df = df[["dataset","groupHPD","stay_id"]].drop_duplicates()
# print(df.groupby(["dataset"]).agg({"stay_id":"count"}))

# 生成特征

In [3]:
df_first = df_[df_["time"]== min(df_["time"])]
df_first.columns = [col + "_first" for col in df_first.columns]
df_first.index = pd.RangeIndex(len(df_first.index))

df_last = df_[df_["time"]== max(df_["time"])]
df_last.columns = [col + "_last" for col in df_last.columns]
df_last.index = pd.RangeIndex(len(df_last.index))

df_grouped = df_.groupby(["stay_id"]).agg(["max","min","mean"])
df_grouped.columns = ['_'.join(col) for col in df_grouped.columns]
df_grouped = df_grouped.reset_index()
df_grouped = pd.concat([df_grouped, df_last,df_first], axis=1)
df_grouped = df_grouped.drop(['time_max','time_min', 'time_mean','stay_id_last', 'time_last', 'stay_id_first','time_first'],axis=1)

tmp_df = df[["stay_id","groupHPD","dataset"]].drop_duplicates()
tmp_df.index = pd.RangeIndex(len(tmp_df.index))
df_grouped = pd.concat([tmp_df[["groupHPD","dataset"]],df_grouped],axis=1)
df_grouped.to_csv("df_saki_self_generate_features_b21a1234.csv",index=False)
df_grouped.shape

(8866, 133)

# 过滤特征

In [4]:
df_fea = pd.read_csv("./df_saki_self_generate_features_b21a1234.csv")
X = df_fea.drop(["stay_id","dataset",'groupHPD'],axis=1)
y = df_fea[["groupHPD"]]
X.shape

(8866, 130)

## person相关系数 特征之间的线性相关

In [5]:
import seaborn as sns
corr_df=X.corr()
threshold = 0.9 # 剔除相关性系数高于threshold的corr_drop
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool)) #返回上三角矩阵
corr_drop = [column for column in upper.columns if any(upper[column].abs() > threshold)]
diff_fea = list(set(X.columns.tolist()).difference(corr_drop))
print("person相关系数大于0.9的特征:",len(corr_drop), "剩余特征数：", len(diff_fea))
X = X[diff_fea]

person相关系数大于0.9的特征: 21 剩余特征数： 109


<ipython-input-5-c0e409dc9687>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool)) #返回上三角矩阵


## 信息量：可以通过计算某个特征对于分类这样的事件到底有多大信息量贡献
- 特征与标签类的互信息越大，特征与标签“相关性越强”。反之。若两者互信息为0,说明两者独立，没什么关系。 
- 互信息在已经给定另一个特征的情况下一个特征可以提供多少信息量。互信息会通过计算两个特征所共有的信息，把上述推理工程形式化表达出来。与相关性不同，它依赖的不是数据序列，而是数据的分布

In [6]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.datasets import load_iris
mic_res = mutual_info_classif(X,y,random_state=10)
XT = X.T
XT["MIC"] = mic_res
XT = XT.sort_values("MIC",ascending=False)
XT = XT[XT["MIC"]>0]
XT = XT.drop(["MIC"],axis=1)
XTT = XT.T
df_final = pd.concat([df_fea[["stay_id","dataset",'groupHPD']],XTT],axis=1)

print("互信息筛选后的特征数量：",XTT.shape[1])

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


互信息筛选后的特征数量： 100


In [7]:
df_final.to_csv("df_saki_self_generate_features_b21a1234_CorrMICfilt.csv",index=False)